In [23]:
import json
import pandas as pd
import numpy as np
import os

In [46]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

In [ ]:
#CREATE COLLECTOR DF
    # READ CSV FROM FILE
    # APPEND TO COLLECTOR DF

# CLEANUP AND ASSESS


In [ ]:
# FUNCTIONS

In [2]:
def assemble_csv(in_dir):
    """
    Reads all CSV files from the specified directory and loads them into a Pandas DataFrame.

    Args:
        in_dir (str): The input directory containing the CSV files.

    Returns:
        A Pandas DataFrame containing the concatenated data from all CSV files.
    """
    # Create an empty list to store the data frames from all CSV files
    df_list = []

    # Iterate over all files in the input directory
    for filename in os.listdir(in_dir):
        if filename.endswith('.csv'):
            # Read the CSV file into a Pandas DataFrame
            df = pd.read_csv(os.path.join(in_dir, filename))

            # Append the data frame to the list
            df_list.append(df)

    # Concatenate all data frames into a single data frame
    combined_df = pd.concat(df_list)

    # Return the combined data frame
    return combined_df


In [5]:
in_dir = 'data/batches_classified'
df = assemble_csv(in_dir)
df.head()

,kw,rank,link,link.1,success,word_count,percent_human,percent_ai
0,rainbow bridge poem,1,https://www.rainbowsbridge.com/poem.htm,https://www.rainbowsbridge.com/poem.htm,True,597.0,99.7,0.3
1,rainbow bridge poem,2,https://petheavencemetery.com/the-rainbow-brid...,https://petheavencemetery.com/the-rainbow-brid...,True,58.0,0.0,0.0
2,rainbow bridge poem,3,https://www.vetwest.com.au/pet-library/rainbow...,https://www.vetwest.com.au/pet-library/rainbow...,True,240.0,94.3,5.7
3,rainbow bridge poem,4,https://www.pinterest.com/pin/841750986579853559/,https://www.pinterest.com/pin/841750986579853559/,False,NaN,NaN,NaN
4,rainbow bridge poem,5,https://www.scrapbook.com/poems/doc/2907.html,https://www.scrapbook.com/poems/doc/2907.html,True,221.0,100.0,0.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3690 entries, 0 to 99
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   kw             3690 non-null   object 
 1   rank           3690 non-null   int64  
 2   link           3690 non-null   object 
 3   link.1         3690 non-null   object 
 4   success        3690 non-null   bool   
 5   word_count     2952 non-null   float64
 6   percent_human  2952 non-null   float64
 7   percent_ai     2952 non-null   float64
dtypes: bool(1), float64(3), int64(1), object(3)
memory usage: 234.2+ KB


In [8]:
df = df.drop(['link.1'], axis=1)

In [9]:
df

,kw,rank,link,success,word_count,percent_human,percent_ai
0,rainbow bridge poem,1,https://www.rainbowsbridge.com/poem.htm,True,597.0,99.7,0.3
1,rainbow bridge poem,2,https://petheavencemetery.com/the-rainbow-brid...,True,58.0,0.0,0.0
2,rainbow bridge poem,3,https://www.vetwest.com.au/pet-library/rainbow...,True,240.0,94.3,5.7
3,rainbow bridge poem,4,https://www.pinterest.com/pin/841750986579853559/,False,NaN,NaN,NaN
4,rainbow bridge poem,5,https://www.scrapbook.com/poems/doc/2907.html,True,221.0,100.0,0.0
...,...,...,...,...,...,...,...
95,drain hose washing machine,16,https://www.geapplianceparts.com/store/parts/s...,True,314.0,21.7,78.3
96,drain hose washing machine,17,https://www.frigidaireapplianceparts.com/Shop-...,True,437.0,65.9,34.1
97,drain hose washing machine,18,https://www.bobvila.com/articles/how-to-drain-...,True,988.0,96.0,4.0
98,drain hose washing machine,19,https://www.ahs.com/home-matters/quick-tips/st...,True,1414.0,82.3,17.7


In [10]:
non_null_count = df['word_count'].count()
print("Number of non-null values in 'word_count':", non_null_count)

Number of non-null values in 'word_count': 2952


In [12]:
true_count = df['success'].sum()
print("Number of True values in 'success':", true_count)

Number of True values in 'success': 2952


In [13]:
# TEXT TOO SHORT

count = ((df['success'] == True) & (df['percent_human'] == 0) & (df['percent_ai'] == 0)).sum()
print("Number of rows where 'success' is True and 'percent_human' and 'percent_ai' are both 0:", count)


Number of rows where 'success' is True and 'percent_human' and 'percent_ai' are both 0: 327


In [14]:
# WHERE % HUMAN AND % AI ARE BOTH 0 THE WORD_COUNT WAS TOO SHORT
short_df = df[(df['success'] == True) & (df['percent_human'] == 0) & (df['percent_ai'] == 0)]S
short_df

,kw,rank,link,success,word_count,percent_human,percent_ai
1,rainbow bridge poem,2,https://petheavencemetery.com/the-rainbow-brid...,True,58.0,0.0,0.0
15,rainbow bridge poem,16,https://www.ebay.com/itm/324565196337,True,14.0,0.0,0.0
36,shea butter,17,https://www.okaypurenaturals.com/shea-butter/,True,26.0,0.0,0.0
50,hoya rope,11,https://www.brsucculentco.com/products/hindu-r...,True,84.0,0.0,0.0
52,hoya rope,13,https://www.plantsandpilatays.com/shop/p/hoya-...,True,61.0,0.0,0.0
...,...,...,...,...,...,...,...
39,heat resistant gloves,20,https://www.bedbathandbeyond.com/store/s/heat-...,True,42.0,0.0,0.0
84,drain hose washing machine,5,https://www.walmart.com/c/kp/washing-machine-d...,True,8.0,0.0,0.0
87,drain hose washing machine,8,https://www.ebay.com/b/Washing-Machine-Drain-H...,True,13.0,0.0,0.0
89,drain hose washing machine,10,https://www.sears.com/search=flexible%20washin...,True,10.0,0.0,0.0


In [74]:
# Assuming you have a Pandas DataFrame named df
data_df = df[(df['percent_human'] + df['percent_ai'] >= 99)]
data_df

,kw,rank,link,success,word_count,percent_human,percent_ai
0,rainbow bridge poem,1,https://www.rainbowsbridge.com/poem.htm,True,597.0,99.7,0.3
2,rainbow bridge poem,3,https://www.vetwest.com.au/pet-library/rainbow...,True,240.0,94.3,5.7
4,rainbow bridge poem,5,https://www.scrapbook.com/poems/doc/2907.html,True,221.0,100.0,0.0
5,rainbow bridge poem,6,https://www.dailypaws.com/living-with-pets/pet...,True,1088.0,99.9,0.1
8,rainbow bridge poem,9,https://en.wikipedia.org/wiki/Rainbow_Bridge_(...,True,769.0,99.9,0.1
...,...,...,...,...,...,...,...
95,drain hose washing machine,16,https://www.geapplianceparts.com/store/parts/s...,True,314.0,21.7,78.3
96,drain hose washing machine,17,https://www.frigidaireapplianceparts.com/Shop-...,True,437.0,65.9,34.1
97,drain hose washing machine,18,https://www.bobvila.com/articles/how-to-drain-...,True,988.0,96.0,4.0
98,drain hose washing machine,19,https://www.ahs.com/home-matters/quick-tips/st...,True,1414.0,82.3,17.7


In [17]:
# Assuming you have a Pandas DataFrame named df
new_df = df[(df['percent_human'] + df['percent_ai'] >= 99)]
percentage = round((len(new_df) / len(df)) * 100, 1)
print("Percentage of rows where the sum of 'percent_human' and 'percent_ai' is 99 or greater:", percentage, "%")


Percentage of rows where the sum of 'percent_human' and 'percent_ai' is 99 or greater: 71.1 %


In [69]:
def count_occurrences(df):
    result = df.groupby('kw').apply(lambda x: ((x['percent_human'] + x['percent_ai']) >= 99).sum()).reset_index(name='count')
    avg_values = df.groupby('kw').agg({'percent_human': [lambda x: np.nan if x.isnull().all() or (x == 0).all() else x.min(),
                                                       lambda x: np.nan if x.isnull().all() or (x == 0).all() else x.mean(),
                                                       lambda x: np.nan if x.isnull().all() or (x == 0).all() else x.max()],
                                       'percent_ai': [lambda x: np.nan if x.isnull().all() or (x == 0).all() else x.min(),
                                                      lambda x: np.nan if x.isnull().all() or (x == 0).all() else x.mean(),
                                                      lambda x: np.nan if x.isnull().all() or (x == 0).all() else x.max()]}).reset_index()
    avg_values.columns = ['kw', '%_Human_Min', '%_Human_Mean', '%_Human_Max', '%_AI_Min', '%_AI_Mean', '%_AI_max']
    avg_values = avg_values.round({'%_Human_Min': 1, '%_Human_Mean': 1, '%_Human_Max': 1, '%_AI_Min': 1, '%_AI_Mean': 1, '%_AI_max': 1})
    result = pd.merge(result, avg_values, on='kw')
    return result

In [70]:
z = count_occurrences(df)
z

,kw,count,%_Human_Min,%_Human_Mean,%_Human_Max,%_AI_Min,%_AI_Mean,%_AI_max
0,ab lounge,9,0.0,64.0,100.0,0.0,5.2,27.0
1,aklief cream,14,0.0,62.6,97.8,0.0,30.8,97.4
2,all the queen's men,9,0.0,67.8,99.4,0.0,14.0,51.8
3,baby blanket,0,NaN,NaN,NaN,NaN,NaN,NaN
4,baby nail clippers,13,0.0,62.6,100.0,0.0,9.6,58.2
5,baby's growth chart,9,55.5,83.4,100.0,0.0,16.6,44.5
6,babygirl nursery,15,0.0,78.7,99.9,0.0,15.1,70.1
7,backyard decor ideas,16,73.9,97.2,100.0,0.0,2.8,26.1
8,beacon orthopaedics & sports medicine,8,0.0,57.4,100.0,0.0,4.2,28.2
9,begonia rex,17,0.0,76.7,99.7,0.0,17.7,72.5


In [75]:
z.to_csv('preliminary_results.csv', index=False)